<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span></font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

## Подготовка

In [1]:
import pandas as pd
#import torch
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
#import transformers
#from tqdm import notebook
import re
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import spacy
import numpy
import nltk 
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline


In [2]:
from nltk.stem.wordnet import WordNetLemmatizer
#from pywsd.utils import lemmatize_sentence
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#from gensim.utils import lemmatize

In [3]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [4]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
df1=df.sample(70000) #выборка слишком большая

In [7]:
df1['text'] = df1['text'].str.lower() # к нижнему регистру


In [9]:
#очистка текста с помощью регулярных выражений
text_new = []
for i in df1.text:
    clear = re.sub(r'[^a-zA-Z0-9]', ' ', i)
    text_new.append(' '.join(clear.split()))

In [10]:
df1['text_clear'] = text_new

In [11]:
df1.head()

,text,toxic,text_clear
144924,"wrong. if the leader disconnects, then the per...",0,wrong if the leader disconnects then the perso...
121204,no tell the real name. i cant think of it. is ...,1,no tell the real name i cant think of it is th...
22638,are there many books on latham yet?,0,are there many books on latham yet
109535,"""\n\nfuck off. wp:mosislam mentions """"pbuh"""" a...",1,fuck off wp mosislam mentions pbuh and as not ...
63527,"""===pathetic settler===\n\nthat's what you see...",1,pathetic settler that s what you seem to be i ...


In [12]:
stop_words = set(stopwords.words('english'))

In [13]:
def lemmatize_text(df):
    ''' функция выполняет токенизациию и лемматизацию массива текстов '''
    L = WordNetLemmatizer()
    df_new = []
    for sentence in df:
        df_new.append(' '.join([L.lemmatize(w) for w in nltk.word_tokenize(sentence) if not w in stopwords.words('english')]))
    return df_new

In [14]:
df1['lemma'] = lemmatize_text(df1['text_clear'])

In [15]:
df1.head()

,text,toxic,text_clear,lemma
144924,"wrong. if the leader disconnects, then the per...",0,wrong if the leader disconnects then the perso...,wrong leader disconnect person joined first le...
121204,no tell the real name. i cant think of it. is ...,1,no tell the real name i cant think of it is th...,tell real name cant think name mental disorder...
22638,are there many books on latham yet?,0,are there many books on latham yet,many book latham yet
109535,"""\n\nfuck off. wp:mosislam mentions """"pbuh"""" a...",1,fuck off wp mosislam mentions pbuh and as not ...,fuck wp mosislam mention pbuh title idiot king...
63527,"""===pathetic settler===\n\nthat's what you see...",1,pathetic settler that s what you seem to be i ...,pathetic settler seem glad silly as upset feel...


In [17]:
features_train, features_test, target_train, target_test = train_test_split(df1['lemma'], df1['toxic'],
                                                                            test_size=0.3, random_state=12345)

In [18]:
print(features_train.shape, features_test.shape, target_train.shape, target_test.shape)

(49000,) (21000,) (49000,) (21000,)


In [20]:
tf_idf = TfidfVectorizer(stop_words = stop_words)
tf_idf_train = tf_idf.fit_transform(features_train)

In [21]:
tf_idf_test = tf_idf.transform(features_test)

**Выводы:**
* данные загружены
* для уменьшения времени обработки решено уменьшить выборку
* данные очищены, токенизированы и лемматизированы
* поделены на обучающую и тестовую выборки
* текст векторизирован

## Обучение

In [23]:
model = LogisticRegression(C=6, class_weight='balanced', solver = 'lbfgs', max_iter = 1000)
model.fit(tf_idf_train, target_train)
prediction = model.predict(tf_idf_test)
f1_lr = f1_score(target_test, prediction)

print(f1_lr)

0.7566480446927373


In [24]:
pipe_lr = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                ('lgb', LGBMClassifier(random_state=12345))])


In [25]:
param = {'lgb__max_depth': range (-1, 0, 10), 'lgb__n_estimators': range (100, 500, 1000), 'lgb__class_weight':['balanced'],
         'lgb__learning_rate':[0.1, 1]}

In [26]:
LG = GridSearchCV(estimator=pipe_lr,
            param_grid=param,
            scoring='f1',
            cv=3) 

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(df1['lemma'], df1['toxic'],
                                                                            test_size=0.3, random_state=12345)

In [28]:
%%time
LG.fit(X_train, Y_train)
predict = LG.predict(X_test)


CPU times: user 14min 56s, sys: 5.84 s, total: 15min 2s
Wall time: 15min 11s


In [29]:
LG.best_score_

0.7276846527394888

In [30]:
f1_lgm = f1_score(Y_test, predict)
print(f1_lgm)

0.7421572840567255


## Выводы

* данные загружены
* для уменьшения времени обработки решено уменьшить выборку
* данные очищены, токенизированы и лемматизированы
* поделены на обучающую и тестовую выборки
* текст векторизирован
* обучены модели
* наилучшее значение у модели LogisticRegression